# Домашнее задание № 1
# "Классификация: Логистическая регрессия и SVM"

#### Имеются данные adult.csv (см. в материалах к занятию).
#### Целевой переменной является уровень дохода income (крайний правый столбец).
#### Описание признаков можно найти по ссылке http://www.cs.toronto.edu/~delve/data/adult/adultDetail.html
#### Вам необходимо построить модели логистической регрессии и SVM, которые предсказывает уровень дохода человека.
#### Вывести качество полученных моделей на тестовой выборке, используя функцию score у модели.
#### Готовый ноутбук выложить на гитхаб и прислать ссылку.

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv( 'D:/DATA_SCIENCE/dataset/dz_1_feml_3/adult.csv' )
data.tail()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
48837,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48838,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
48839,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
48840,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K
48841,52,Self-emp-inc,287927,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024,0,40,United-States,>50K


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


### Попробуем предсказать доход на основе возраста, рассы, пола
### Проверим пустые значения в признаках

In [5]:
data[ pd.isnull( data['age'] ) ].head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income


In [6]:
print('Для age пустых строк {}'.format( len( data[ pd.isnull( data['age'] ) ] ) ))
print('Для race пустых строк {}'.format( len( data[ pd.isnull( data['race'] ) ] ) ))
print('Для gender пустых строк {}'.format( len( data[ pd.isnull( data['gender'] ) ] ) ))
print('Всего строк в наборе {}'.format( len( data ) ))

Для age пустых строк 0
Для race пустых строк 0
Для gender пустых строк 0
Всего строк в наборе 48842


### 1. Построение модели логистической регрессии


In [7]:
from sklearn.linear_model import LogisticRegression

In [8]:
# Выбираем признаки
selectedColumns = data[ [ 'age', 'race', 'gender', 'income' ] ]

In [9]:
# столбец race является категориальной переменной
# переведем ее в значения 0 и 1, добавив столбцы с соответствующими названиями
X = pd.get_dummies( selectedColumns, columns = [ 'race' ] )

In [10]:
#  переменная пол(столбец gender) является категориальной
# переведем значения столбца в числа
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [11]:
le.fit( data['gender'] )

LabelEncoder()

In [12]:
le.classes_

array(['Female', 'Male'], dtype=object)

In [13]:
# Расшировка столбца gender
le.transform( [ 'Male', 'Female', 'Male' ] )

array([1, 0, 1])

In [14]:
# записываем в переменную X преобразованный столбец gender
X.gender = le.transform(data['gender'] ) 
X

,age,gender,income,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White
0,25,1,<=50K,0,0,1,0,0
1,38,1,<=50K,0,0,0,0,1
2,28,1,>50K,0,0,0,0,1
3,44,1,>50K,0,0,1,0,0
4,18,0,<=50K,0,0,0,0,1
...,...,...,...,...,...,...,...,...
48837,27,0,<=50K,0,0,0,0,1
48838,40,1,>50K,0,0,0,0,1
48839,58,0,<=50K,0,0,0,0,1
48840,22,1,<=50K,0,0,0,0,1


In [15]:
# столбец income является целевой переменной, удаляем его из X
del X['income']
X.head()

,age,gender,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White
0,25,1,0,0,1,0,0
1,38,1,0,0,0,0,1
2,28,1,0,0,0,0,1
3,44,1,0,0,1,0,0
4,18,0,0,0,0,0,1


In [16]:
# целевая переменная (столбец income) является категориальной
# переведем значения столбца в числа

In [17]:
le.fit( data['income'] )

LabelEncoder()

In [18]:
le.classes_

array(['<=50K', '>50K'], dtype=object)

In [19]:
# Расшировка столбца income

le.transform( [ '<=50K', '>50K', '<=50K' ] )

array([0, 1, 0])

In [20]:
# записываем в переменную y преобразованный столбец income

y = pd.Series( data = le.transform( data['income'] ) )
y.head()

0    0
1    0
2    1
3    1
4    0
dtype: int32

In [21]:
model = LogisticRegression()

In [22]:
X

,age,gender,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White
0,25,1,0,0,1,0,0
1,38,1,0,0,0,0,1
2,28,1,0,0,0,0,1
3,44,1,0,0,1,0,0
4,18,0,0,0,0,0,1
...,...,...,...,...,...,...,...
48837,27,0,0,0,0,0,1
48838,40,1,0,0,0,0,1
48839,58,0,0,0,0,0,1
48840,22,1,0,0,0,0,1


In [23]:
# Разделяем данные на тестовые и тренировочные
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [24]:
# Обучаем модель

model.fit(X_train, y_train )
predictions = model.predict_proba(X_test)

In [25]:
predictions[:5]

array([[0.72257558, 0.27742442],
       [0.54096692, 0.45903308],
       [0.61451841, 0.38548159],
       [0.56893982, 0.43106018],
       [0.94771846, 0.05228154]])

In [26]:
y_pred = model.predict(X_test)

In [27]:
# Оцениваем верность модели на тестовых данных
model.score(X_test, y_test)

0.7448304101549171

In [28]:
from sklearn.metrics import accuracy_score
print('Верность: %.2f' % accuracy_score(y_test, y_pred))

Верность: 0.74


### 2. Построение модели SVM

In [38]:
from sklearn.svm import SVC
model_svm = SVC(kernel='linear', C=1.0, random_state=0)
model_svm.fit(X_train, y_train)

SVC(random_state=0)

In [39]:
y_pred = model_svm.predict(X_test)

In [40]:
# Оцениваем верность модели на тестовых данных
model_svm.score(X_test, y_test)

0.7601173821060534

In [41]:
from sklearn.metrics import accuracy_score
print('Верность: %.2f' % accuracy_score(y_test, y_pred))

Верность: 0.76
